# Практическое задание 1
**Определение любимого напитка человека методом _k_ ближайших соседей**

1. Конвертируем полученные с помощью опроса данные из XLSX в CSV при помощи https://cloudconvert.com/xlsx-to-csv

Выберем первые 5 элементов (голосов):

In [2]:
import pandas as pd
try:
    data = pd.read_csv('./tea-vs-coffee.csv', engine='python')
except:
    data = pd.read_csv('https://github.com/dmittrj/AIST_Practice_1/blob/main/tea-vs-coffee.csv', engine='python')
data.head(10)

,Опрашиваемый,Укажите Ваш пол,Какой напиток Вы предпочитаете утром?,Укажите Ваш возраст,Насколько здоровый образ жизни Вы ведете? \nУкажите число по шкале от 0 до 100,Курите ли Вы?,Укажите цвет Вашего левого глаза,Много ли Вы испытываете стресса в жизни? Укажите число от 0 до 100,Вы высыпаетесь?,Укажите Ваш хронотип,"Во сколько Вы обычно просыпаетесь? Укажите время в формате ""Час"" и ""Минуты""",Сколько Вы в среднем спите? Укажите среди время вашего сна,Если поблизости с Вашим домом кофейня?,Вы являетесь гурманом?,Вы работаете из офиса?,Вы домосед?,У Вас есть хронические заболевания?,Какой рукой Вы пишите?,Какой у Вас знак зодиака?
0,1,Женщина,Кофе,22,65,Да,Серо-зеленый,55,Да,Жаворонок,8:00,8.0,Да,Да,Да,Нет,Нет,Правой,Козерог
1,2,Мужчина,Чай,21,85,Нет,Зеленый,70,Да,Жаворонок,6:00,8.0,Да,Нет,Нет,Нет,Нет,Левой,Стрелец
2,3,Мужчина,Кофе,22,50,Да,Голубой,70,Да,Сова,9:00,7.0,Да,Да,Нет,Да,Нет,Правой,Рак
3,4,Мужчина,Кофе,22,80,Да,Карий,60,Да,Сова,9:00,6.0,Да,Да,Нет,Нет,Да,Правой,Скорпион
4,5,Женщина,Кофе,23,50,Нет,Голубой,90,Нет,Сова,14:00,6.0,Нет,Нет,Нет,Да,Да,Левой,Скорпион
5,6,Мужчина,Чай,21,99,Нет,Зелёный,1,Да,Жаворонок,6:30,8.0,Нет,Да,Нет,Нет,Нет,Правой,Овен
6,7,Мужчина,Чай,22,50,Нет,Голубой,70,Да,Сова,10:00,6.0,Нет,Нет,Нет,Нет,Нет,Правой,Рыбы
7,8,Мужчина,Чай,22,80,Нет,Голубой,45,Да,Сова,7:00,7.0,Да,Нет,Да,Нет,Да,Правой,Водолей
8,9,Мужчина,Кофе,22,50,Нет,Коричневый,70,Нет,Сова,0:00,8.0,Да,Да,Нет,Да,Нет,Левой,Дева
9,10,Мужчина,Чай,21,70,Да,Зеленый,100,Нет,Сова,0:30,6.0,Нет,Нет,Нет,Да,Нет,Правой,Водолей
